In [1]:
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(data.table))


In [3]:
in_f <- '17ihiw-Unrelated-FQ.tsv'

In [4]:
out_f <- '17ihiw-Unrelated-FQ.2digits.tsv'

In [5]:
df <- fread(in_f)

In [24]:
df %>%
head()%>%
rename(!!str_replace(names(df)[1], '#', '') := names(df)[1] ) %>%
rename(group = names(df)[3]) %>%
mutate(
    Allele_locus = str_replace(Allele, '\\*[0-9:]+', ''),
    Allele_digits = str_replace(Allele, sprintf('%s\\*', Allele_locus), '')
) %>%
mutate(
    Allele = sprintf('%s*%s', Allele_locus, str_sub(Allele_digits, 1, 2)),
) %>%
select(-Allele_locus, -Allele_digits) %>%
group_by(Locus, group, Allele) %>%
summarise_at(names(df)[4:ncol(df)], sum, na.rm = TRUE)


Locus,group,Allele,Frequency,Count
<chr>,<chr>,<chr>,<dbl>,<int>
HLA-A,Global,A*01,0.13362,1155
HLA-A,Global,A*02,0.23311,2015
HLA-A,Global,A*03,0.10805,934
HLA-A,Global,A*11,0.06571,568
HLA-A,Global,A*24,0.08017,693
HLA-A,Global,A*32,0.03702,320


In [75]:
df %>%
rename(!!str_replace(names(df)[1], '#', '') := names(df)[1] ) %>%
rename(group = names(df)[3]) %>%
separate(Allele, c('Allele_locus', 'Allele_digits'), sep='\\*') %>%
mutate(
    Allele = sprintf('%s*%s', Allele_locus, str_sub(Allele_digits, 1, 2)),
) %>%
select(-Allele_locus, -Allele_digits) %>%
group_by(Locus, group, Allele) %>%
summarise(
    Count = sum(Count),
    Frequency = sum(Frequency)
) %>%
ungroup() %>%
rename(!!names(df)[3] := 'group') %>%
rename(!!names(df)[1] := str_replace(names(df)[1], '#', '')) %>%
select(names(df)) %>%
fwrite(out_f, sep='\t', na = "NA", quote=F)
